# Construisons GPT à partir de rien 

Ce notebook va présenter la création à partir de rien d'un modèle de langage pour prédire le prochain caractère qui se base sur l'architecture du transformers (encodeur en particulier).  
Pour cela, nous utilons un fichier texte moliere.txt qui regroupe l'intégralité des dialogues des pièces de Molière.   
Ce dataset a été crée à partir des oeuvres complètes de Molière disponibles sur le site [Gutenberg.org](Gutenberg.org). J'ai nettoyé un peu les données pour ne garder que les dialogues.

## Lecture du dataset

Commençons par ouvrir et par visualiser un peu ce que contient notre dataset.

In [2]:
with open('moliere.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("Nombre de caractères dans le dataset : ", len(text))

Nombre de caractères dans le dataset :  1687290


Affichons les 250 premiers caractères : 

In [4]:
print(text[:250])

VALÈRE.

Eh bien, Sabine, quel conseil me donnes-tu?

SABINE.

Vraiment, il y a bien des nouvelles. Mon oncle veut résolûment que ma
cousine épouse Villebrequin, et les affaires sont tellement avancées,
que je crois qu'ils eussent été mariés dès aujo


Utilisons set() pour récuperer les caractères uniques présent dans le dataset.

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print("Nombre de caractères différents : ", vocab_size)


 !'(),-.:;?ABCDEFGHIJKLMNOPQRSTUVXYZabcdefghijlmnopqrstuvxyz«»ÇÈÉÊÏàâæçèéêëìîïòôùûŒœ
Nombre de caractères différents :  85


## Création de notre dataset d'entraînement

Comme dans le cours 5, nous allons créer un maping pour passer de caractères à entier. Le mapping que nous faisons ici est une forme de tokenization la plus simple possible.   

### Point rapide sur la tokenization

**La tokenization, qu'est ce que c'est ?** : La tokenization est le processus de conversion d'un texte en séquence d'entier où chaque entier peut correspondre à un caractère, un groupe de caractère ou un mot selon les méthodes employées.   

**Balance entre Vocabulaire et taille de séquence** : Un bon tokenizer trouve une balance entre la taille du vocabulaire (26 pour toutes les lettres de l'alphabet et ~100 000 pour les nombre de mots de la langue française). Plus on a une taille de vocabulaire petite, plus les séquences seront longues (le mot "Bonjour" est encodé par 7 tokens si notre vocabulaire est au niveau du caractère et un seul token si notre vocabulaire regroupe tous les mots de la langue française) et inversement. En pratique, les deux extrèmes sont problémiques et on cherche le juste milieu.  

**Tokenizer de la littérature** : Les tokenizers sont une part importante du bon fonctionnement d'un modèle de langage. 
La façon de créer un bon tokenizer dépend de la méthode et des données d'entraînement. Parmi les tokenizers les plus utilisés, on retrouve [SentencePiece](https://github.com/google/sentencepiece) de Google et [tiktoken](https://github.com/openai/tiktoken) de OpenAI.


In [6]:
# Creation d'un mapping de caractère à entiers et inversement
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encore : prend un string et output une liste d'entiers
decode = lambda l: ''.join([itos[i] for i in l]) # decode: prend une liste d'entiers et output un string

print(encode("Bonjour à tous"))
print(decode(encode("Bonjour à Tous")))

[13, 50, 49, 46, 50, 56, 53, 1, 68, 1, 55, 50, 56, 54]
Bonjour à Tous


On va transformer notre dataset en séquence d'entier et le stocker sous forme de tenseur pytorch.

In [7]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:250]) # Les 250 premiers caractères encodé

tensor([33, 12, 23, 64, 29, 16,  8,  0,  0, 16, 44,  1, 38, 45, 41, 49,  6,  1,
        30, 37, 38, 45, 49, 41,  6,  1, 52, 56, 41, 47,  1, 39, 50, 49, 54, 41,
        45, 47,  1, 48, 41,  1, 40, 50, 49, 49, 41, 54,  7, 55, 56, 11,  0,  0,
        30, 12, 13, 20, 25, 16,  8,  0,  0, 33, 53, 37, 45, 48, 41, 49, 55,  6,
         1, 45, 47,  1, 59,  1, 37,  1, 38, 45, 41, 49,  1, 40, 41, 54,  1, 49,
        50, 56, 57, 41, 47, 47, 41, 54,  8,  1, 24, 50, 49,  1, 50, 49, 39, 47,
        41,  1, 57, 41, 56, 55,  1, 53, 73, 54, 50, 47, 82, 48, 41, 49, 55,  1,
        52, 56, 41,  1, 48, 37,  0, 39, 50, 56, 54, 45, 49, 41,  1, 73, 51, 50,
        56, 54, 41,  1, 33, 45, 47, 47, 41, 38, 53, 41, 52, 56, 45, 49,  6,  1,
        41, 55,  1, 47, 41, 54,  1, 37, 42, 42, 37, 45, 53, 41, 54,  1, 54, 50,
        49, 55,  1, 55, 41, 47, 47, 41, 48, 41, 49, 55,  1, 37, 57, 37, 49, 39,
        73, 41, 54,  6,  0, 52, 56, 41,  1, 46, 41,  1, 39, 53, 50, 45, 54,  1,
        52, 56,  3, 45, 47, 54,  1, 41, 

On va maintenant découper notre texte en une partie training et une partie validation. Prenons un ratio de 0.9-0.1.

In [8]:
n = int(0.9*len(data)) # 90% pour le train et 10% pour la validation
train_data = data[:n]
val_data = data[n:]

Pour notre modèle de langage, on va également définir une taille de contexte "block_size".

In [14]:
block_size = 8
train_data[:block_size+1]

tensor([33, 12, 23, 64, 29, 16,  8,  0,  0])

Ici, les 8 premiers caractères represente le contexte et le 9ème est le label. Nous verrons ensuite que ce simple exemple regroupe en fait une multitude d'exemples.   
En effet, notre modèle doit être capable de prédire le prochain caractère peu importe le contexte qu'il a en amont. On a donc, dans cette liste, 8 exemples qui sont les suivants :  

In [15]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"Quand l'entrée est {context.numpy()} le label est : {target}")

Quand l'entrée est [33] le label est : 12
Quand l'entrée est [33 12] le label est : 23
Quand l'entrée est [33 12 23] le label est : 64
Quand l'entrée est [33 12 23 64] le label est : 29
Quand l'entrée est [33 12 23 64 29] le label est : 16
Quand l'entrée est [33 12 23 64 29 16] le label est : 8
Quand l'entrée est [33 12 23 64 29 16  8] le label est : 0
Quand l'entrée est [33 12 23 64 29 16  8  0] le label est : 0


On sait maintenant comme créer un ensemble de entrée/label à partir d'un seul exemple. Adaptons cette méthode pour un traitement en batch : 

In [20]:
batch_size = 4 # La taille de batch (les séquences calculés en parallèles)
block_size = 8 # La taille de contexte maximale pour une prédiction du modèle

def get_batch(split):
    # On genere un batch de données (sur train ou val)
    data = train_data if split == 'train' else val_data
    #On génére batch_size indice de début de séquence pris au hasard dans le dataset
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # On stocke dans notre tenseur torch
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('Entrée : ')
print(xb.shape)
print(xb)
print('Labels :')
print(yb.shape)
print(yb)

Entrée : 
torch.Size([4, 8])
tensor([[ 0,  0, 30, 18, 12, 25, 12, 29],
        [40, 37, 49, 55,  1, 52, 56, 41],
        [56, 53,  1, 12, 47, 38, 41, 53],
        [37, 45, 54,  1, 47, 68,  1, 40]])
Labels :
torch.Size([4, 8])
tensor([[ 0, 30, 18, 12, 25, 12, 29, 16],
        [37, 49, 55,  1, 52, 56, 41,  1],
        [53,  1, 12, 47, 38, 41, 53, 55],
        [45, 54,  1, 47, 68,  1, 40, 41]])


Chacun de ces 4 exemples regroupe 8 exemples distincts (comme expliqué précedemment), cela fait donc un total de 32 exemples. 

## Performance du bigramme

Dans le cours 5 sur les NLP, nous avons vu le bigramme qui peut être considéré comme le modèle de langage le plus simple et qui consiste à prédire la prochain caractère à partir d'un unique caractère de contexte. Notons $B$ pour la batch_size, $T$ pour le block_size et $C$ pour le vocab_size.    
Pour voir sa performance sur le dataset moliere.txt, implémentons le rapidement en pytorch :  

In [27]:
import torch
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    # Chaque token va directement lire la valeur du prochain à partir d'une look-up table entrainé
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):
    # Taille (B,T)
    logits = self.token_embedding_table(idx) 
    # Taille (B,T,C)
    
    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    # idx est de la taille (B,T) avec T le contexte actuel
    for _ in range(max_new_tokens):
      # Forward du modèle pour récuperer les prédictions
      logits, loss = self(idx)
      # On prend uniquement le dernier caractère
      logits = logits[:, -1, :] # devient (B, C)
      # On applique la softmax pour récuperer les probabilités
      probs = F.softmax(logits, dim=-1) # (B, C)
      # On sample avec torch.multinomial
      idx_next = torch.multinomial(probs, num_samples=1) # devient (B, 1)
      # On ajouter l'élément sample à la séquence actuelle
      idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 85])
tensor(5.1318, grad_fn=<NllLossBackward0>)


Le modèle est implementé mais non entraîné, si on le teste comme ça on obtient des résultats catastrophiques : 

In [30]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


ScgI:MœUSÊgtrYZc(-DêDm)(Œ'T;jàSduÏâTMtUæòUHjOôbMrVEmà
h'sfîRH»Dîxd;ŒhZê(âMoJjVoŒMmdZùï.tgîlDpr» CæŒK


C'est tout simplement aléatoire et c'est logique car le modèle est initialisé aléatoirement.  
On va maintenant entrainer le modèle :  

In [34]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
batch_size = 32
steps=10000
for step in range(steps): # Nombre d'étape d'entraînement (élements traités = steps*batch_size)

    # On récupère un batch de données aléatoires
    xb, yb = get_batch('train')

    # On calcule le loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    # Retropropagation
    loss.backward()
    # Mise à jour des poids du modèle
    optimizer.step()

print(loss.item())

2.342592477798462


Générons à partir de notre modèle entrainé : 

In [37]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=300)[0].tolist()))


t t joieçolèr TTE.

DIluptrr dias me men, qul e? l feut an, à pastirou GOtrs ve ames à p
ELE.
Pas atesunt-vofroitur j'y trte delace ét mplant ce padi quendene t, apagi mittere côte cl de che.

JUn eloin jeuene me IRSONEtren.
Qu pun di asque e,

ARÉLVaiens l pçuntonceuem'euss vonde.
Ahoil'ablà au'avo


On constate une amélioration dans la structuration des données et certains mots semblent presque correct mais ça reste catastrophique. En soit, on s'attendait à ce résultat car le bigramme est un modèle trop simple. 